# Fine Tuning Open AI Models

Dis notebook dey base on di current guidance wey dey for [Fine Tuning](https://platform.openai.com/docs/guides/fine-tuning?WT.mc_id=academic-105485-koreyst) documentation from Open AI.

Fine-tuning dey help improve how foundation models go perform for your application by retrain am with extra data and context wey dey relevant to di specific use case or scenario. Make you sabi say prompt engineering techniques like _few shot learning_ and _retrieval augmented generation_ fit help you add better data to di default prompt to make di quality better. But, dis methods get limit because of di max token window size wey di foundation model fit handle.

With fine-tuning, we dey retrain di model itself with di data wey we need (dis one go allow us use plenty examples wey pass di max token window size) - and we go deploy one _custom_ version of di model wey no go need examples again during inference time. Dis one no just dey improve how we fit design prompt (we go get more space to use di token window for other things) but e fit also reduce our cost (because we no go need send plenty tokens to di model during inference time).

Fine tuning get 4 steps:
1. Prepare di training data and upload am.
1. Run di training job to get di fine-tuned model.
1. Check di fine-tuned model and adjust am to make sure e dey okay.
1. Deploy di fine-tuned model for inference when you don satisfy.

Make you sabi say no be all foundation models dey support fine-tuning - [check OpenAI documentation](https://platform.openai.com/docs/guides/fine-tuning/what-models-can-be-fine-tuned?WT.mc_id=academic-105485-koreyst) to see di latest information. You fit also fine-tune model wey dem don fine-tune before. For dis tutorial, we go use `gpt-35-turbo` as di foundation model wey we wan fine-tune.

---


### Step 1.1: Prepare Your Dataset

Make we build one chatbot wey go help you sabi di periodic table of elements by ansa question about one element wit limerick. For _dis_ simple tutorial, we go just create one dataset to train di model wit small sample examples of response wey go show di format wey di data suppose get. For real-life use, you go need create one dataset wey get plenty examples. You fit also use open dataset (wey match your application area) if e dey, and reformat am to use for fine-tuning.

Since we dey focus on `gpt-35-turbo` and we dey look for single-turn response (chat completion), we fit create examples using [dis format wey dem suggest](https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset?WT.mc_id=academic-105485-koreyst) wey match di OpenAI chat completion requirements. If you dey expect multi-turn conversation content, you go use di [multi-turn example format](https://platform.openai.com/docs/guides/fine-tuning/multi-turn-chat-examples?WT.mc_id=academic-105485-koreyst) wey get `weight` parameter to show which message dem go use (or no go use) for di fine-tuning process.

For dis tutorial, we go use di simple single-turn format. Di data dey for [jsonl format](https://jsonlines.org/?WT.mc_id=academic-105485-koreyst) wey get 1 record per line, each one na JSON-formatted object. Di snippet wey dey below show 2 records as sample - check [training-data.jsonl](../../../../../18-fine-tuning/python/openai/training-data.jsonl) for di full sample set (10 examples) wey we go use for di fine-tuning tutorial. **Note:** Each record _must_ dey for one single line (e no suppose dey split across lines like di normal formatted JSON file).

```json
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Gallium"}, {"role": "assistant", "content": "Gallium, oh gallium, so light - Melts in your hand, oh what a sight - At 86 degrees - Its liquid with ease - And in semiconductors, it's out of sight"}]}
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Hydrogen"}, {"role": "assistant", "content": "Hydrogen, the first in the line - The lightest of all, so divine - It's in water, you see - And in stars, it's the key - The universe's most common sign"}]}
```

For real-life use, you go need plenty examples to get better results - di tradeoff na between di quality of di response and di time/money wey e go cost for fine-tuning. We dey use small set so we fit finish di fine-tuning quick to show di process. Check [dis OpenAI Cookbook example](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_finetune_chat_models.ipynb?WT.mc_id=academic-105485-koreyst) for one more complex fine-tuning tutorial.


---

### Step 1.2 Upload Your Dataset

Make you upload di data wit di Files API [as dem explain am here](https://platform.openai.com/docs/guides/fine-tuning/upload-a-training-file). Remember say to fit run dis code, you go need don do dis steps first:
 - Install di `openai` Python package (make sure say you dey use version wey be >=0.28.0 to get di latest features)
 - Set di `OPENAI_API_KEY` environment variable to your OpenAI API key
To sabi more, check di [Setup guide](./../../../00-course-setup/02-setup-local.md?WT.mc_id=academic-105485-koreyst) wey dem provide for di course.

Now, run di code to create file wey you go upload from your local JSONL file.


In [24]:
from openai import OpenAI
client = OpenAI()

ft_file = client.files.create(
  file=open("./training-data.jsonl", "rb"),
  purpose="fine-tune"
)

print(ft_file)
print("Training File ID: " + ft_file.id)

FileObject(id='file-JdAJcagdOTG6ACNlFWzuzmyV', bytes=4021, created_at=1715566183, filename='training-data.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)
Training File ID: file-JdAJcagdOTG6ACNlFWzuzmyV


---

### Step 2.1: Create di Fine-tuning job wit di SDK


In [25]:
from openai import OpenAI
client = OpenAI()

ft_filejob = client.fine_tuning.jobs.create(
  training_file=ft_file.id, 
  model="gpt-3.5-turbo"
)

print(ft_filejob)
print("Fine-tuning Job ID: " + ft_filejob.id)

FineTuningJob(id='ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', created_at=1715566184, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-EZ6ag0n0S6Zm8eV9BSWKmE6l', result_files=[], seed=830529052, status='validating_files', trained_tokens=None, training_file='file-JdAJcagdOTG6ACNlFWzuzmyV', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)
Fine-tuning Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh


### Step 2.2: Check di Status of di job

Dis na some things wey you fit do wit di `client.fine_tuning.jobs` API:
- `client.fine_tuning.jobs.list(limit=<n>)` - Show di last n fine-tuning jobs
- `client.fine_tuning.jobs.retrieve(<job_id>)` - See di details of one fine-tuning job
- `client.fine_tuning.jobs.cancel(<job_id>)` - Stop one fine-tuning job
- `client.fine_tuning.jobs.list_events(fine_tuning_job_id=<job_id>, limit=<b>)` - Show up to n events wey happen for di job
- `client.fine_tuning.jobs.create(model="gpt-35-turbo", training_file="your-training-file.jsonl", ...)`

Di first step for di process na _to check di training file_ to make sure say di data dey correct format.


In [26]:
from openai import OpenAI
client = OpenAI()

# List 10 fine-tuning jobs
client.fine_tuning.jobs.list(limit=10)

# Retrieve the state of a fine-tune
client.fine_tuning.jobs.retrieve(ft_filejob.id)

# List up to 10 events from a fine-tuning job
client.fine_tuning.jobs.list_events(fine_tuning_job_id=ft_filejob.id, limit=10)

SyncCursorPage[FineTuningJobEvent](data=[FineTuningJobEvent(id='ftevent-GkWiDgZmOsuv4q5cSTEGscY6', created_at=1715566184, level='info', message='Validating training file: file-JdAJcagdOTG6ACNlFWzuzmyV', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-3899xdVTO3LN7Q7LkKLMJUnb', created_at=1715566184, level='info', message='Created fine-tuning job: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', object='fine_tuning.job.event', data={}, type='message')], object='list', has_more=False)

In [30]:
# Once the training data is validated
# Track the job status to see if it is running and when it is complete
from openai import OpenAI
client = OpenAI()

response = client.fine_tuning.jobs.retrieve(ft_filejob.id)

print("Job ID:", response.id)
print("Status:", response.status)
print("Trained Tokens:", response.trained_tokens)

Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh
Status: running
Trained Tokens: None


### Step 2.3: Dey track events to check progress


In [44]:
# You can also track progress in a more granular way by checking for events
# Refresh this code till you get the `The job has successfully completed` message
response = client.fine_tuning.jobs.list_events(ft_filejob.id)

events = response.data
events.reverse()

for event in events:
    print(event.message)

Step 85/100: training loss=0.14
Step 86/100: training loss=0.00
Step 87/100: training loss=0.00
Step 88/100: training loss=0.07
Step 89/100: training loss=0.00
Step 90/100: training loss=0.00
Step 91/100: training loss=0.00
Step 92/100: training loss=0.00
Step 93/100: training loss=0.00
Step 94/100: training loss=0.00
Step 95/100: training loss=0.08
Step 96/100: training loss=0.05
Step 97/100: training loss=0.00
Step 98/100: training loss=0.00
Step 99/100: training loss=0.00
Step 100/100: training loss=0.00
Checkpoint created at step 80 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyyF2:ckpt-step-80
Checkpoint created at step 90 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyzhK:ckpt-step-90
New fine-tuned model created: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz
The job has successfully completed


### Step 2.4: Check status for OpenAI Dashboard


You fit see check di status by go di OpenAI website and look di _Fine-tuning_ section for di platform. Dis one go show you di status of di current job, and e go still allow you track di history of di jobs wey dem don run before. For dis screenshot, you go see say di job wey dem run before fail, but di second one work well. To explain wetin happen, di first run bin use JSON file wey get wrong format for di records - but afta dem fix am, di second run finish well and di model come dey ready to use.

![Fine-tuning job status](../../../../../translated_images/fine-tuned-model-status.563271727bf7bfba7e3f73a201f8712fae3cea1c08f7c7f12ca469c06d234122.pcm.png)


You fit see di status messages and metrics if you scroll go down for di visual dashboard as e dey show:

| Messages | Metrics |
|:---|:---|
| ![Messages](../../../../../translated_images/fine-tuned-messages-panel.4ed0c2da5ea1313b3a706a66f66bf5007c379cd9219cfb74cb30c0b04b90c4c8.pcm.png) |  ![Metrics](../../../../../translated_images/fine-tuned-metrics-panel.700d7e4995a652299584ab181536a6cfb67691a897a518b6c7a2aa0a17f1a30d.pcm.png)|


---

### Step 3.1: Collect ID & Test Fine-Tuned Model for Code


In [46]:
# Retrieve the identity of the fine-tuned model once ready
response = client.fine_tuning.jobs.retrieve(ft_filejob.id)
fine_tuned_model_id = response.fine_tuned_model
print("Fine-tuned Model ID:", fine_tuned_model_id)

Fine-tuned Model ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz


In [47]:
# You can then use that model to generate completions from the SDK as shown
# Or you can load that model into the OpenAI Playground (in the UI) to validate it from there.
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model=fine_tuned_model_id,
  messages=[
    {"role": "system", "content": "You are Elle, a factual chatbot that answers questions about elements in the periodic table with a limerick"},
    {"role": "user", "content": "Tell me about Strontium"},
  ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content="Strontium, a metal so bright - It's in fireworks, a dazzling sight - It's in bones, you see - And in tea, it's the key - It's the fortieth, so pure, that's the right", role='assistant', function_call=None, tool_calls=None)


### Step 3.2: Load & Test Fine-Tuned Model for Playground

You fit test di fine-tuned model for two ways now. First, you fit go di Playground and use di Models drop-down to choose your new fine-tuned model from di options wey dey. Di other way na to use di "Playground" option wey dey di Fine-tuning panel (check di screenshot above) wey go open di _comparative_ view wey go show di foundation and fine-tuned model versions side-by-side so you fit check am quick.

![Fine-tuning job status](../../../../../translated_images/fine-tuned-playground-compare.56e06f0ad8922016497d39ced3d84ea296eec89073503f2bf346ec9718f913b5.pcm.png)

Just put di system context wey you use for your training data and add your test question. You go see say both sides go update with di same context and question. Run di comparison and you go see di difference for di outputs between dem. _Notice how di fine-tuned model dey give response for di format wey you provide for your examples, while di foundation model dey just follow di system prompt_.

![Fine-tuning job status](../../../../../translated_images/fine-tuned-playground-launch.5a26495c983c6350c227e05700a47a89002d132949a56fa4ff37f266ebe997b2.pcm.png)

You go see say di comparison go also show di token counts for each model, and di time wey e take for di inference. **Dis example na simple one wey dey show di process but e no dey reflect real world dataset or situation**. You fit notice say both samples dey show di same number of tokens (system context and user prompt dey di same) but di fine-tuned model dey take more time for inference (custom model).

For real-world situations, you no go dey use example wey be like dis one, but you go dey fine-tune with real data (e.g., product catalog for customer service) wey go make di response quality clear well. For _dat_ kind situation, to get di same response quality with di foundation model go need more custom prompt engineering wey go increase token usage and fit add di processing time for inference. _To try am, check di fine-tuning examples for di OpenAI Cookbook to start._


<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Disclaimer**:  
Dis dokyument don use AI transle-shon service [Co-op Translator](https://github.com/Azure/co-op-translator) do di transle-shon. Even as we dey try make am accurate, abeg make you sabi say transle-shon wey machine do fit get mistake or no dey correct well. Di original dokyument for di language wey dem write am first na di one wey you go take as di correct source. For important mata, e good make you use professional human transle-shon. We no go fit take blame for any misunderstanding or wrong interpretation wey fit happen because you use dis transle-shon.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
